In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pkg_resources

from debvader.training.train import train_deblender

2024-11-05 02:57:53.090302: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 02:57:53.114469: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-05 02:57:53.626640: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Download and format data

Download dataset that will be used for training. It has been generated using the code in https://github.com/BastienArcelin/dc2_img_generation and the stamps are generated using the XXX function see notebook XXX. The size of the stamp is then fixed to 59x59 pixels.

In [2]:
data_folder_path = pkg_resources.resource_filename('debvader', "data/")
image_path = os.path.join(data_folder_path + '/dc2_imgs/imgs_dc2.npy')
images = np.load(image_path, mmap_mode = 'c')

In [3]:
images.shape

(10, 59, 59, 6)

Separate training data and labels to feed the VAE and deblender. You should be careful that the number of filter in the data for trainng correspond to the number of bands considered for the network (default is six).

In [6]:
training_data_vae = np.array((images[:5], images[:5]))
validation_data_vae = np.array((images[5:], images[5:]))

training_data_deblender = np.array((images[:5], images[:5]))
validation_data_deblender = np.array((images[5:], images[5:]))

### Train the VAE and deblender with architecture from the debvader paper (https://arxiv.org/abs/2005.12039)

In [7]:
hist_vae, hist_deblender, net = train_deblender("lsst",
                                                      from_survey = "dc2", 
                                                      epochs = 2, 
                                                      training_data_vae = training_data_vae, 
                                                      validation_data_vae = validation_data_vae, 
                                                      training_data_deblender = training_data_deblender, 
                                                      validation_data_deblender = validation_data_deblender)

in cropping
VAE model
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 59, 59, 6)]       0         
                                                                 
 model_4 (Functional)        (None, 560)               3741224   
                                                                 
 multivariate_normal_tri_l_  ((None, 32),              0         
 1 (MultivariateNormalTriL)   (None, 32))                        
                                                                 
 model_5 (Functional)        (None, 59, 59, 6)         4577228   
                                                                 
Total params: 8318452 (31.73 MB)
Trainable params: 8318440 (31.73 MB)
Non-trainable params: 12 (48.00 Byte)
_________________________________________________________________
/home/parlange/debvader/src/debvader/data/weights/dc2

Start 

### Dataset with a different number of filters

The number of available filters is different for each survey. For example, five filters are available for DES data. To change that you should specify it in the train_deblender function, as well as if the channels appear last or first in the data array.

In [8]:
training_data_vae.shape

(2, 5, 59, 59, 6)

Here the channels appear last in the data array. So the channel_first option in the train_deblender function is set as False (default setting).

In [11]:
hist_vae, hist_deblender, net = train_deblender("des",
                                                      from_survey = "dc2", 
                                                      epochs = 2, 
                                                      training_data_vae = training_data_vae, 
                                                      validation_data_vae = validation_data_vae, 
                                                      training_data_deblender = training_data_deblender, 
                                                      validation_data_deblender = validation_data_deblender,
                                                      nb_of_bands = 6,
                                                      channel_last = True)

in cropping
VAE model
Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 59, 59, 6)]       0         
                                                                 
 model_16 (Functional)       (None, 560)               3741224   
                                                                 
 multivariate_normal_tri_l_  ((None, 32),              0         
 4 (MultivariateNormalTriL)   (None, 32))                        
                                                                 
 model_17 (Functional)       (None, 59, 59, 6)         4577228   
                                                                 
Total params: 8318452 (31.73 MB)
Trainable params: 8318440 (31.73 MB)
Non-trainable params: 12 (48.00 Byte)
_________________________________________________________________
/home/parlange/debvader/src/debvader/data/weights/dc2

Start

We need to change the training data format. For example just train on the five fist channels.

In [12]:
training_data_vae_deslike = np.array((images[:5,:,:,:5], images[:5,:,:,:5]))
validation_data_vae_deslike = np.array((images[5:,:,:,:5], images[5:,:,:,:5]))

training_data_deblender_deslike = np.array((images[:5,:,:,:5], images[:5,:,:,:5]))
validation_data_deblender_deslike = np.array((images[5:,:,:,:5], images[5:,:,:,:5]))

Now try the training:

Warning: we cannot use the weights from the network trained on DC2 as the data has six channels in that case. We need to set the from_survey option to None.

In [13]:
hist_vae, hist_deblender, net = train_deblender("des",
                                                      from_survey = None, 
                                                      epochs = 2, 
                                                      training_data_vae = training_data_vae_deslike, 
                                                      validation_data_vae = validation_data_vae_deslike, 
                                                      training_data_deblender = training_data_deblender_deslike, 
                                                      validation_data_deblender = validation_data_deblender_deslike,
                                                      nb_of_bands = 5,
                                                      channel_last = True)

in cropping
VAE model
Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 59, 59, 5)]       0         
                                                                 
 model_20 (Functional)       (None, 560)               3740932   
                                                                 
 multivariate_normal_tri_l_  ((None, 32),              0         
 5 (MultivariateNormalTriL)   (None, 32))                        
                                                                 
 model_21 (Functional)       (None, 59, 59, 5)         4576650   
                                                                 
Total params: 8317582 (31.73 MB)
Trainable params: 8317572 (31.73 MB)
Non-trainable params: 10 (40.00 Byte)
_________________________________________________________________

Start the training
Epoch 1/2
1/1 - 2s - loss: 1949626.7500 